In [1]:
# import dependencies
import os
import io
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime
from pytz import timezone
import pandas as pd
import FoodTools

In [2]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [3]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [4]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [5]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)
print('Length: ', len(meal_type))

['Breakfast', 'Lunch', 'Dinner']
Length:  3


In [6]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)
print('Length: ', len(food_list))

['Banana and Coconut Pancakes', 'Belgian Waffles', 'Cage Free Scrambled Eggs', 'Coconut Syrup', 'Hard-boiled Cage Free Eggs', 'Natural BridgesTofu Scramble', 'Oatmeal Gluten-Free', 'Sausage Patties', 'Shredded Hash Browns', 'Chocolate Muffin', 'Nutella Hazelnut Cheese Ring', 'Oatbran Apple Muffin', 'Vegan Peanut Granola', 'Bell Peppers', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cheddar Cheese', 'Feta Cheese', 'Fresh Organic Spinach', 'Fresh Tomato', 'Jalapeno', 'Onions', 'Oven Roasted Turkey', 'Sliced Olived', 'Smoked Ham', 'N.E.Clam Chowder', 'Tuscan Vegetable Soup', 'Baked Salmon with Avocado Chimichurri', 'Black Rice, Beet and Kale Salad', 'Oven Roasted Chicken Thigh', 'Sesame Chili Ginger Beef', 'Sesame Chow Mein Noodles', 'Steamed Rice', 'Szechuan Sweet Chili Tofu', 'Cheese Pizza', 'Roasted Seasonal Vegetables', 'Mint Chip Brownie', "Paul's Vegan Macadamia Nut Cookies", 'BAR Wings', 'BBQ Wings', 'Condiments', 'Hawaiian Coleslaw', "Hot 'N Spicy Wings", 'Teriyaki Glaze Wings', '

In [7]:
# In case of dupliate food name, the list should be converted to a set
# sort
food_list = sorted(list(set(food_list)))
print(food_list)
print('Sorted length: ', len(food_list))

['BAR Wings', 'BBQ Wings', 'Baked Salmon with Avocado Chimichurri', 'Banana and Coconut Pancakes', 'Bar Indian', 'Basmati Rice Pullao', 'Belgian Waffles', 'Bell Peppers', 'Black Rice, Beet and Kale Salad', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cage Free Scrambled Eggs', 'California Pasta', 'Cheddar Cheese', 'Cheese Pizza', 'Chocolate Muffin', 'Coconut Syrup', 'Condiments', 'Dal Saag', 'Feta Cheese', 'Fresh Organic Spinach', 'Fresh Tomato', 'Hard-boiled Cage Free Eggs', 'Hawaiian Coleslaw', 'Heath Bar Almond Pie', "Hot 'N Spicy Wings", 'Jalapeno', 'Masala Vegetables', 'Mint Chip Brownie', 'N.E.Clam Chowder', 'Natural BridgesTofu Scramble', 'Nutella Hazelnut Cheese Ring', 'Oatbran Apple Muffin', 'Oatmeal Gluten-Free', 'Onions', 'Original Naan', 'Oven Roasted Chicken Thigh', 'Oven Roasted Turkey', "Paul's Vegan Macadamia Nut Cookies", 'Roasted Seasonal Vegetables', 'Sausage Patties', 'Sesame Chili Ginger Beef', 'Sesame Chow Mein Noodles', 'Shredded Hash Browns', 'Sliced Olived', 'S

In [8]:
# preference list, labeled by myself
# pref_1 = [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0 ]
# pref_2 = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
# pref_3 = [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
# pref_4 = [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
# pref_5 = [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
pref = []
for i in food_list:
    print(i)
    preference = int(input())
    if preference != 0 and preference != 1:
        print('Incrorrect input')
        break
    pref.append(preference)
print(pref)
print('Preference length: ', len(pref))

BAR Wings
0
BBQ Wings
0
Baked Salmon with Avocado Chimichurri
0
Banana and Coconut Pancakes
1
Bar Indian
0
Basmati Rice Pullao
0
Belgian Waffles
0
Bell Peppers
0
Black Rice, Beet and Kale Salad
0
Cage Free Eggs
0
Cage Free Omelette Bar
0
Cage Free Scrambled Eggs
1
California Pasta
1
Cheddar Cheese
0
Cheese Pizza
1
Chocolate Muffin
0
Coconut Syrup
0
Condiments
0
Dal Saag
0
Feta Cheese
0
Fresh Organic Spinach
0
Fresh Tomato
0
Hard-boiled Cage Free Eggs
0
Hawaiian Coleslaw
0
Heath Bar Almond Pie
0
Hot 'N Spicy Wings
0
Jalapeno
0
Masala Vegetables
0
Mint Chip Brownie
0
N.E.Clam Chowder
0
Natural BridgesTofu Scramble
0
Nutella Hazelnut Cheese Ring
0
Oatbran Apple Muffin
0
Oatmeal Gluten-Free
0
Onions
0
Original Naan
0
Oven Roasted Chicken Thigh
0
Oven Roasted Turkey
1
Paul's Vegan Macadamia Nut Cookies
0
Roasted Seasonal Vegetables
0
Sausage Patties
1
Sesame Chili Ginger Beef
1
Sesame Chow Mein Noodles
1
Shredded Hash Browns
0
Sliced Olived
0
Smoked Ham
0
Steamed Rice
1
Szechuan Sweet Chili

In [9]:
# timezone setting
us_pacific = timezone('US/Pacific')
time = datetime.now(us_pacific)
us_time = time.strftime('%Y-%m-%d')
print('Current US/Pacific time: ', us_time)

Current US/Pacific time:  2018-09-21


In [10]:
# write the food list to a csv file
path = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/{}.csv'.format(us_time)
print('Current path: ', path)
path_2 = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/'

Current path:  /Users/7w0r4ng3s/Desktop/menu_scraping/data/2018-09-21.csv


In [11]:
def write_data():
    with open(path, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for key, val in zip(food_list, pref):
            writer.writerow([key, val])
    print('write_csv: COMPLETED')
    # add index and column names
    df = pd.read_csv(path, names=['food', 'pref'])
    df.index.names = ['index']
    df.to_csv(path)
    print('add_column_name: COMPLETED')
    
write_data()

write_csv: COMPLETED
add_column_name: COMPLETED


In [12]:
# def merge_data():
#     # TODO: Figure out a way to get rid of the unnamed: 0 column
#     # TODO: Modify merge_data() so that new data can be append to data.csv
#     files = [f for f in os.listdir('.') if os.path.isfile(f)]

#     merged = []

#     for f in files:
#         filename, ext = os.path.splitext(f)
#         if ext == '.csv':
#             read = pd.read_csv(f)
#             merged.append(read)

#     result = pd.concat(merged)
#     result.to_csv('data.csv')
    
# merge_data()

In [13]:
def append_data():
    df1 = pd.read_csv('Data.csv', index_col='index')
    df1.index.names = ['index']
    df2 = pd.read_csv(path, index_col='index')
    df3 = df1.append(df2).reset_index()
    df3 = df3.drop('index', 1).sort_values('food').reset_index().drop('index', 1)
    df3.index.names = ['index']
    df3.to_csv('Data.csv')
    
append_data()

In [18]:
FoodTools.search()

Food name: chili
Do you mean:
1 BBQ Tofu Chili Garlic
2 Chili con Carne
3 Szechuan Sweet Chili Sauce

Enter the index: 3
--------------------------------------------------------------------------------
Out of 2 time(s), you like [Szechuan Sweet Chili Sauce] 2 time(s).
--------------------------------------------------------------------------------


1.0